- logger:是我们进行日志记录时创建的对象，我们可以调用它的方法传入日志模板和信息，来生成一条条日志记录
- log record:就代指生成的一条条日志记录
- handler:即用来处理日志记录的类，它可以将log record输出到我们指定的日志位置和存储形式等 ，如我们可以指定将日志通过FTP协议记录到远程的服务器上
- formatter:实际上生成的log record也是一个个对象，那么我们想要把它们保存一条条我们想要的日志文件的话就需要一个格式化的过程，那么这个过程就由formatter来完成，返回的就是日志字符串
- filter：另外保存日志的时候我们可能不需要全部保存，我们可能只需要保存我们想要的部分就可以了，所以保存前还需要进行一下过滤，比如只留下我们想要的某个级别的日志，那么这个过滤的过程的就应该交给filter来做
- parent handler：handler之间可以存在分层关系，以使得不同的handler之间共享相同功能的代码

### log相对于print的优势在哪些
- 可以设置日志等级，在不同的版本上通过设置不同的输出等级来记录对应的日志，非常灵活
- 可以设置输出到任意位置，如写入文件，写入远程服务器等
- 具有灵活的配置和格式化功能，如配置输出当前模块信息、运行时间等

### 一个简单的实例
- 通过basicConfig配置了level信息和format信息，这里的level配置为INFO信息，即只输出INFO信息。 另外这里指定了format格式的字符串，包括asctime,name,levelname,message 四个内容，分别代运行时间，模块名称，日志级别，日志内容，这样输出内容便是这四者组合而成的内容了

- 接下来申明了一个logger对象，它就是日志输出的主类，调用对象的info()方法就可以输出info级别的日志信息
- 为什么输出的时候只有三条日志信息，每条日志信息都是对应了指定的格式化内容，但是debug的内容没有输出，这是因为我们在全局配置的时候设置了输出INFO级别，所以debug级别的信息就被过滤掉了
- 

In [1]:
import logging
logging.basicConfig(level=logging.DEBUG,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

logger.info('THIS is a log info')
logger.debug('Debugging')
logger.warning('warning message')
logger.info('finish')

2018-08-25 09:55:04,084 - __main__ - INFO - THIS is a log info
2018-08-25 09:55:04,104 - __main__ - DEBUG - Debugging
2018-08-25 09:55:04,106 - __main__ - WARNING - warning message
2018-08-25 09:55:04,109 - __main__ - INFO - finish


### 下面详解一下basicConfig中的一些参数
- filename：即日志输出的文件名，如果指定了这个信息之后，实际上会启用filehandler，而不是streamHandler，这样输出日志就将会输出到文件当中
- filemode: 这个是指定日志文件的写入方式，有两种方式，w和a，分别代表的是清除后写入和追加写入
- format： 指定日志文件的输出格式，即上文中所述的参数，部分参数如下：
    - %(levelno)s:打印日志级别的数值
    - %(levelname)s:打印日志级别的名称
    - %(pathname)s:打印当前<font face="微软雅黑"> 执行程序 </font>的路径，其实就是sys.argv[0]
    - %(filename)s:打印当前执行执行程序名
    - %(funcName)s:打印日志的当前函数
    - %(lineno)d:打印日志的当前行号
    - %(asctime)s:打印日志的时间 
    - %(thread)d:打印线程ID
    - %(threadName)s:打印线程名称
    - %(message)s:
    - %(module)s:
- datefmt:指定时间的输出格式
- style：如果format参数指定了，这个参数就可以指定格式化时的占位符风格，如%，｛，$等 
- stream: 在没有设定filename的时候会默认使用streamhandler，这时stream可以指定初始化的文件流
- handlers:可以指定日志处理时所使用的handlers，必须是可迭代的

In [3]:
import sys
sys.argv[0]

'H:\\anaconda\\lib\\site-packages\\ipykernel_launcher.py'

#### 再用一个简单的实例
- 这里我们指定输出文件的名称为output.log
- 另外指定了日期的输出格式，其中年月日格式改变了
- 另外输出的format格式增加了lineno、module这两个信息，运行之后便会生成一个output.log的文件

In [1]:
import logging
logging.basicConfig(level = logging.DEBUG,
                   filename = 'output.log',
                   datefmt = '%Y/%m/%d %H:%M:%S',
                   format = '%(asctime)s - %(name)s - %(levelname)s - %(lineno)d - %(module)s - %(message)s')
logger = logging.getLogger(__name__)
logger.info('this is a log info')
logger.debug('Debugging')
logger.warning('Warning exists')
logger.info('Finish')

### 更加详细的了解一下level模块
每个等级其实对应一个数值
- CRITICAL 50
- FATAL 50
- ERROR 40
- WARNING 30
- WARN 30
- INFO 20
- DEBUG 10
- NOTSET 0

这里最高等级的是CRITICAL和FATAL，两个对应的数值都是50，另外对于WARNING还提供了简写形式WARN，两个对应的数值都是30
我们设置level，系统便只会输出level数值大于或等于该level的日志结果

### 关于handler的用法
- 这里我们先申明了一个logger对象，然后指定了其对应的handler为对应的filehandler对象，然后handler对象还单独指定了formatter对象单独配置输出格式，最后给logger对象添加对应的handler即可，最后可以发现日志就会被输出到文件中
- logging模块的handler有：
    - StreamHandler:日志输出到流，可以是sys.stderr,sys.stdout或者文件
    - FileHandler:日志输出到文件
    - BaseRotatingHandler:logging.handlers.BaseRotatingHandler 基本日志的回滚方式
    - RotatingHandler:logging.handlers.RotatingHandler 日志回滚方式，支持日志文件最大数量和日志文件回滚
    - TimeRotatingHandler:logging.handlers.TimeRotatingHandler 日志回滚方式，在一定时间区域内回滚日志文件
    - SocketHandler:logging.handlers.SocketHandler 远程输出日志到TCP/IP sockets
    - DatagramHandler
    - SMTPHandler:logging.handlers.SMTPHandler 远程输出日志到邮件地址
    - SysLogHandler
    - HTTPHandler

In [2]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.INFO)
handler = logging.FileHandler('output.log')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(lineno)d - %(module)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

logger.info('This is a log info')
logger.debug('Debugging')
logger.warning('Warning exists')
logger.info('Finish')

In [3]:
import logging
from logging.handlers import HTTPHandler
import sys

logger = logging.getLogger(__name__)
logger.setLevel(level = logging.DEBUG)
http_handler = HTTPHandler(host = 'localhost:8001',url = 'log',method = 'POST')
logger.addHandler(http_handler)

logger.info('This is a log info')
logger.debug('Debugging')
logger.warning('Warning exists')
logger.info('Finish')

--- Logging error ---
Traceback (most recent call last):
  File "H:\anaconda\lib\logging\handlers.py", line 1198, in emit
    h.endheaders()
  File "H:\anaconda\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "H:\anaconda\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "H:\anaconda\lib\http\client.py", line 964, in send
    self.connect()
  File "H:\anaconda\lib\http\client.py", line 936, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "H:\anaconda\lib\socket.py", line 724, in create_connection
    raise err
  File "H:\anaconda\lib\socket.py", line 713, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Call stack:
  File "H:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "H:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "H:\anacond

--- Logging error ---
Traceback (most recent call last):
  File "H:\anaconda\lib\logging\handlers.py", line 1198, in emit
    h.endheaders()
  File "H:\anaconda\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "H:\anaconda\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "H:\anaconda\lib\http\client.py", line 964, in send
    self.connect()
  File "H:\anaconda\lib\http\client.py", line 936, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "H:\anaconda\lib\socket.py", line 724, in create_connection
    raise err
  File "H:\anaconda\lib\socket.py", line 713, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Call stack:
  File "H:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "H:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "H:\anacond

In [4]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(level=logging.DEBUG)

# Formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# FileHandler
file_handler = logging.FileHandler('result.log')
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)

# StreamHandler
stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

# Log
logger.info('Start')
logger.warning('Something maybe fail.')
try:
    result = 10 / 0
except Exception:
    logger.error('Faild to get result', exc_info=True)
logger.info('Finished')

--- Logging error ---
Traceback (most recent call last):
  File "H:\anaconda\lib\logging\handlers.py", line 1198, in emit
    h.endheaders()
  File "H:\anaconda\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "H:\anaconda\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "H:\anaconda\lib\http\client.py", line 964, in send
    self.connect()
  File "H:\anaconda\lib\http\client.py", line 936, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "H:\anaconda\lib\socket.py", line 724, in create_connection
    raise err
  File "H:\anaconda\lib\socket.py", line 713, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Call stack:
  File "H:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "H:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "H:\anacond

--- Logging error ---
Traceback (most recent call last):
  File "H:\anaconda\lib\logging\handlers.py", line 1198, in emit
    h.endheaders()
  File "H:\anaconda\lib\http\client.py", line 1234, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "H:\anaconda\lib\http\client.py", line 1026, in _send_output
    self.send(msg)
  File "H:\anaconda\lib\http\client.py", line 964, in send
    self.connect()
  File "H:\anaconda\lib\http\client.py", line 936, in connect
    (self.host,self.port), self.timeout, self.source_address)
  File "H:\anaconda\lib\socket.py", line 724, in create_connection
    raise err
  File "H:\anaconda\lib\socket.py", line 713, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] 由于目标计算机积极拒绝，无法连接。
Call stack:
  File "H:\anaconda\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "H:\anaconda\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "H:\anacond